<a href="https://colab.research.google.com/github/parasgulati8/Web-Scraper/blob/master/scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# import glob
import pandas as pd
import numpy as np
import time
from bs4 import BeautifulSoup as bs
# import urllib3
import requests
# import json
# from urllib.request import urlopen, urlretrieve, quote
# from openpyxl import Workbook
import re

In [0]:
# Property Summary Page
url = 'http://www.padctn.org/prc/property/1/card/0'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
lists = soup.find_all('ul', attrs={'class':'att'})
table = dict()
for i in range(len(lists)):
  lists[i] = lists[i].find_all('li')
  lists[i] = [item.text for item in lists[i]]
  lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
  for pair in lists[i]:
    key , value = pair
    table[key] = value

index = 1
property_summary = pd.DataFrame(table, index=[index])

# Historical data
url = 'http://www.padctn.org/prc/property/1/card/1/historical'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
parcel = soup.h2.text
parcel = ' '.join(parcel.split()[6:])
dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                      attrs={"class":"table table-condensed table-bordered"})
sales_history, previous_appraisals = dfs[0], dfs[1]
sales_history['Map & Parcel'] = [parcel]*len(sales_history)
sales_history['Property #'] = [1]*len(sales_history)
previous_appraisals['Map & Parcel'] = [parcel]*len(previous_appraisals)
previous_appraisals['Property #'] = [1]*len(previous_appraisals)

# adding the parcel to property summary table
if property_summary.iloc[-1, 0] == '':
  property_summary.iloc[-1,0] = parcel

# Building and Improvement Details Page
url = 'http://www.padctn.org/prc/property/1/card/0/interior'
page = requests.get(url)
soup = bs(page.text, 'html.parser')
lists = (soup.find_all('ul', attrs={'class':'att'}))
table = dict()
for i in range(len(lists)):
  lists[i] = lists[i].find_all('li')
  lists[i] = [item.text for item in lists[i]]
  lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
  for pair in lists[i]:
    key , value = pair
    table[key] = value

temp_table = pd.DataFrame(table, index=[0])
temp_table['Map & Parcel'] = [parcel]
dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text)
building_footage, building_features = dfs[0], dfs[1]
building_features['Map & Parcel'] = [parcel]*(1 if len(building_features) == 0 else len(building_features))
building_footage['Map & Parcel'] = [parcel]*len(building_footage)

merged = pd.merge(building_footage, building_features, on='Map & Parcel')
building_improvement = pd.merge(merged, temp_table, on='Map & Parcel')

index = index+1

In [68]:
while True:
  # Property Summary Page
  url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/0'
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  lists = soup.find_all('ul', attrs={'class':'att'})
  table = dict()
  for i in range(len(lists)):
    lists[i] = lists[i].find_all('li')
    lists[i] = [item.text for item in lists[i]]
    lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
    for pair in lists[i]:
      key , value = pair
      table[key] = value
  
  summary = pd.DataFrame(table, index=[index])
  property_summary = pd.concat((property_summary,summary))


  # Historical data
  url = 'http://www.padctn.org/prc/property/' + str(index) + '/card/1/historical'
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  parcel = soup.h2.text[34:]
  
  dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text,
                        attrs={"class":"table table-condensed table-bordered"})
  dfs[0]['Map & Parcel'] = [parcel]*len(dfs[0])
  dfs[0]['Property #'] = [index]*len(dfs[0])
  dfs[1]['Map & Parcel'] = [parcel]*len(dfs[1])
  dfs[1]['Property #'] = [index]*len(dfs[1])

  if property_summary.iloc[-1, 0] == '':
    property_summary.iloc[-1,0] = parcel
  print(index, end=' ')
  if np.any(dfs[0]) or np.any(dfs[1]):
    sales_history = pd.concat([sales_history, dfs[0]])
    previous_appraisals = pd.concat([previous_appraisals, dfs[1]])
  else:
    print ('Table Empty, Exiting...')
    print (table)
    break

  # Building and Improvements Details
  url = 'http://www.padctn.org/prc/property/' + str(index) +'/card/0/interior'
  page = requests.get(url)
  soup = bs(page.text, 'html.parser')
  lists = (soup.find_all('ul', attrs={'class':'att'}))
  table = dict()
  for i in range(len(lists)):
    lists[i] = lists[i].find_all('li')
    lists[i] = [item.text for item in lists[i]]
    lists[i] = [lists[i][j].split(': ') for j in range(len(lists[i]))]
    for pair in lists[i]:
      key , value = pair
      table[key] = value

  temp_table = pd.DataFrame(table, index=[0])
  temp_table['Map & Parcel'] = [parcel]
  dfs = pd.read_html(requests.get(url,  headers={'User-agent': 'Mozilla/5.0'}).text)
  building_footage, building_features = dfs[0], dfs[1]
  building_features['Map & Parcel'] = [parcel]*(1 if len(building_features) == 0 else len(building_features))
  building_footage['Map & Parcel'] = [parcel]*len(building_footage)

  merged = pd.merge(building_footage, building_features, on='Map & Parcel')
  building_details = pd.merge(merged, temp_table, on='Map & Parcel')
  building_improvement = pd.concat([building_improvement, building_details])

  if index % 10 == 1:
    print()
  # time.sleep(3)
  if index == 5:
    break
  index = index+1  

2 3 4 5 

In [69]:
property_summary

,Map & Parcel,Location,Current Owner,Mailing Address,Legal Description,Tax District,Assessment Classification*,Legal Reference,Sale Date,Sale Price,Assessment Year,Last Reappraisal Year,Improvement Value,Land Value,Total Appraisal Value,Assessed Value,Property Use,Zone,Neighborhood,Land Area,Property Type,Year Built,Square Footage,Exterior Wall,Story Height,Building Condition,Foundation Type,Number of Rooms,Number of Beds,Number of Baths,Number of Half Bath,Number of Fixtures
1,001 00 0 001.00,,,", ,",,View Tax Record,,View Deed,,$,2019,2017,$0,"$10,621","$10,621","$2,655",,,,Acres,,,,,,,,,,,,
2,001 00 0 002.00,,,", ,",,View Tax Record,,View Deed,,$,2019,2017,$0,"$16,323","$16,323","$4,081",,,,Acres,,,,,,,,,,,,
3,002 00 0 001.00,,,", ,",,View Tax Record,,View Deed,,$,2019,2017,$0,$0,$0,$0,,,,Acres,,,,,,,,,,,,
4,002 00 0 001.00 001,,,", ,",,View Tax Record,,View Deed,,$,2019,2017,"$34,800","$12,300","$47,100",$0,,,,Acres,,,,,,,,,,,,
5,002 00 0 001.00 002,,,", ,",,View Tax Record,,View Deed,,$,2019,2017,$0,"$27,400","$27,400",$0,,,,Acres,,,,,,,,,,,,


In [70]:
sales_history

,Sale Date,Sale Price,Deed Type,Deed Book & Page,Map & Parcel,Property #
0,12/05/2006,"$7,500",DEED OF CORRECTION,20110805-0060547,001 00 0 001.00,1
1,12/05/2006,"$7,500",WARRANTY DEED,20070109-0003943,001 00 0 001.00,1
2,09/15/1965,$0,WARRANTY DEED,00000137-0000216,001 00 0 001.00,1
0,05/17/1983,"$12,500",WARRANTY DEED,00006058-0000985,001 00 0 002.00,2
1,01/06/1972,$0,WARRANTY DEED,00005797-0000390,001 00 0 002.00,2
2,01/06/1972,$0,WARRANTY DEED,00000155-0000282,001 00 0 002.00,2
0,12/18/1997,"$67,000",WARRANTY DEED,00010729-0000778,002 00 0 001.00,3
1,07/29/1997,"$50,000",WARRANTY DEED,00010552-0000735,002 00 0 001.00,3
2,04/13/1994,"$8,000",WARRANTY DEED,00009312-0000366,002 00 0 001.00,3
3,10/27/1977,"$23,000",WARRANTY DEED,00005215-0000894,002 00 0 001.00,3


In [71]:
previous_appraisals

,Year,Land Use Code,Building,Yard Items,Land Value,Category,Total,Map & Parcel,Property #
0,2017,0AR - AG/RES,$0,$0,"$10,621",INF,"$10,621",001 00 0 001.00,1
1,2017,0AR - AG/RES,$0,$0,"$10,621",ROLL,"$10,621",001 00 0 001.00,1
2,2013,0RA - RES/AGR,$0,$0,"$8,565",INF,"$8,565",001 00 0 001.00,1
3,2013,0RA - RES/AGR,$0,$0,"$8,565",ROLL,"$8,565",001 00 0 001.00,1
4,2009,0RA - RES/AGR,$0,$0,"$6,909",INF,"$6,909",001 00 0 001.00,1
5,2009,0RA - RES/AGR,$0,$0,"$6,909",ROLL,"$6,909",001 00 0 001.00,1
6,2008,0RA - RES/AGR,$0,$0,"$5,572",JUNE,"$5,572",001 00 0 001.00,1
7,2008,0RA - RES/AGR,$0,$0,"$5,572",ROLL,"$5,572",001 00 0 001.00,1
8,2005,E80 - EXE,$0,$0,"$25,800",ROLL,"$25,800",001 00 0 001.00,1
9,2001,E80 - EXE,$0,$0,"$22,800",ROLL,"$22,800",001 00 0 001.00,1


In [72]:
building_improvement

,Sub Area *,Sketched Area,Finished Area,Map & Parcel,Improvement Type,Quantity,Units,Condition,Property Type,Year Built,Story Height,Living Units,Exterior Wall,Building Condition,Foundation Type,Roof Cover,Number of Rooms,Number of Beds,Number of Baths,Number of Half Bath,Number of Fixtures
0,Total,0,0,001 00 0 001.00,NaN,NaN,NaN,NaN,,,,,,,,,,,,,
0,Total,0,0,001 00 0 002.00,NaN,NaN,NaN,NaN,,,,,,,,,,,,,
0,Total,0,0,002 00 0 001.00,NaN,NaN,NaN,NaN,,,,,,,,,,,,,
0,Total,0,0,002 00 0 001.00 001,NaN,NaN,NaN,NaN,,,,,,,,,,,,,
0,Total,0,0,002 00 0 001.00 002,NaN,NaN,NaN,NaN,,,,,,,,,,,,,


In [0]:
property_summary.to_csv('Property Summary.csv')
sales_history.to_csv('Sales History.csv')
previous_appraisals.to_csv('Previous Appraisals.csv')
building_improvement.to_csv('Building Improvement.csv')

In [75]:
ls

'Building Improvement.csv'  'Property Summary.csv'   sample_data/
'Previous Appraisals.csv'   'Sales History.csv'


In [73]:
index

5